In [16]:
import json
import time
import s3fs
import boto3
import pandas as pd

import sagemaker

In [17]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity()["Account"]

s3 = s3fs.S3FileSystem(anon=False)

random_state = 42

Using AWS Region: us-east-1


In [12]:
%store -r
%store

Stored variables and their in-db values:
data_prefix                         -> 'sagemaker-tutorial/data'
default_bucket                      -> 'sagemaker-us-east-1-367158743199'
header                              -> ['LABEL', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIA
hyperparameters                     -> {'max_depth': '5', 'eta': '0.2', 'gamma': '4', 'mi
local_data_dir                      -> '../data'
local_processed_path                -> '../data/df_processed.csv'
local_raw_path                      -> '../data/dataset.csv'
model_2_name                        -> 'sagemaker-tutorial-xgboost-smote'
mp2_arn                             -> 'arn:aws:sagemaker:us-east-1:367158743199:model-pa
mpg_name                            -> 'sagemaker-tutorial'
prefix                              -> 'sagemaker-tutorial'
s3_raw_data                         -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t
test_data_uri                       -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t


# Approve the Second model

In [4]:
second_model_package = sagemaker_boto_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList'][0]
model_package_update = {
    'ModelPackageArn': second_model_package['ModelPackageArn'],
    'ModelApprovalStatus': 'Approved'
}

update_response = sagemaker_boto_client.update_model_package(**model_package_update)

# Create an endpoint config + endpoint

In [13]:
approved_model_name = model_2_name

endpoint_name = f'{approved_model_name}-endpoint'
endpoint_instance_count = 1
endpoint_instance_type = "ml.m4.xlarge"


primary_container = {'ModelPackageName': second_model_package['ModelPackageArn']}
endpoint_config_name=f'{approved_model_name}-endpoint-config'
existing_configs = len(sagemaker_boto_client.list_endpoint_configs(NameContains=endpoint_config_name, MaxResults = 30)['EndpointConfigs'])

if existing_configs == 0:
    create_ep_config_response = sagemaker_boto_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            'InstanceType': endpoint_instance_type,
            'InitialVariantWeight': 1,
            'InitialInstanceCount': endpoint_instance_count,
            'ModelName': approved_model_name,
            'VariantName': 'AllTraffic'
        }]
    )
    %store endpoint_config_name
    %approved_model_name

In [14]:
existing_endpoints = sagemaker_boto_client.list_endpoints(NameContains=endpoint_name, MaxResults = 30)['Endpoints']
if not existing_endpoints:
    create_endpoint_response = sagemaker_boto_client.create_endpoint(
        EndpointName=endpoint_name,
        EndpointConfigName=endpoint_config_name)
    %store endpoint_name

endpoint_info = sagemaker_boto_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = endpoint_info['EndpointStatus']

while endpoint_status == 'Creating':
    endpoint_info = sagemaker_boto_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = endpoint_info['EndpointStatus']
    print('Endpoint status:', endpoint_status)
    if endpoint_status == 'Creating':
        time.sleep(60)



## Create a predictor

In [15]:
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session)

# Make predictions!

### sample a row from the test data

In [31]:
s3.download(test_data_uri, f'{local_data_dir}/test.csv')

[None]

In [32]:
df_test = pd.read_csv(f'{local_data_dir}/test.csv', header=None)
df_test.columns = header

df_test = df_test.drop(columns=['LABEL'])

In [41]:
test_input = ','.join([str(x) for x in df_test.sample(1).values.flatten().tolist()])

In [47]:
results = predictor.predict(test_input, initial_args = {"ContentType": "text/csv"})
prediction = json.loads(results)
prediction
print (f'Probablitity of default is:', prediction)

Probablitity of default is: 0.12812666594982147
